In [1]:
import music21
import midiutil

# Load the MIDI file
midi_file = music21.converter.parse('midi_files/twinkle_easy.mid')

# Convert the MIDI file into notes and chords
notes = []
for element in midi_file.flat.notesAndRests:
    if isinstance(element, music21.note.Note):
        notes.append(element)
    elif isinstance(element, music21.chord.Chord):
        for note in element:
            notes.append(note)

print("There are {} notes in this musical piece".format(len(notes)))
midi_numbers = []
midi_dict = {}

# Display the notes
for note in notes:
    item = music21.pitch.Pitch(note.pitch).midi
    midi_numbers.append(item)
    if item in midi_dict:
      midi_dict[item] += 1
    else:
      midi_dict[item] = 1

There are 100 notes in this musical piece


In [2]:
print(midi_numbers)

[60, 60, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60, 67, 67, 65, 65, 64, 64, 62, 67, 67, 65, 65, 64, 64, 62, 60, 60, 67, 67, 69, 69, 69, 67, 65, 65, 64, 64, 62, 60, 62, 64, 60, 60, 62, 60, 62, 67, 67, 67, 67, 69, 69, 69, 69, 67, 65, 65, 65, 65, 64, 65, 62, 62, 62, 62, 60, 67, 67, 67, 67, 65, 65, 64, 64, 64, 64, 64, 62, 62, 67, 67, 67, 67, 65, 65, 65, 65, 65, 65, 65, 65, 64, 64, 64, 62, 62, 62]


In [3]:
print(midi_dict)

{60: 10, 67: 23, 69: 9, 65: 23, 64: 18, 62: 17}


In [4]:
# We use pyaudio to open the microphone and run aubio.pitch on the stream of
# incoming samples. The script will run until KeyboardInterruptCtrl+C is pressed.

import pyaudio
import sys
import numpy as np
import aubio

# Initialize pyaudio
p = pyaudio.PyAudio()

# Open stream
buffer_size = 1024
pyaudio_format = pyaudio.paFloat32
n_channels = 1
samplerate = 44100
stream = p.open(format=pyaudio_format,
                channels=n_channels,
                rate=samplerate,
                input=True,
                frames_per_buffer=buffer_size)

if len(sys.argv) > 1:
    # record 5 seconds
    output_filename = sys.argv[1]
    record_duration = 5 # exit 1
    outputsink = aubio.sink(sys.argv[1], samplerate)
    total_frames = 0
else:
    # run forever
    outputsink = None
    record_duration = None

# Setup pitch
tolerance = 0.8
win_s = 4096 # fft size
hop_s = buffer_size # hop size
pitch_o = aubio.pitch("default", win_s, hop_s, samplerate)
pitch_o.set_unit("midi")
pitch_o.set_tolerance(tolerance)
temp = -1

print("*** Recording has started - Keep quiet and only play the instrument ***")
while True:
    try:
        audiobuffer = stream.read(buffer_size)
        signal = np.frombuffer(audiobuffer, dtype=np.float32)

        pitch = pitch_o(signal)[0]
        confidence = pitch_o.get_confidence()
        pitch = round(pitch)
        if(temp==pitch):
            continue

        if(pitch>60):
            print("{} / {}".format(pitch,confidence))
            temp = pitch
        if outputsink:
            outputsink(signal, len(signal))

    except KeyboardInterrupt:
        print("*** Ctrl+C pressed ***")
        break

print("*** Recording has ended ***")
stream.stop_stream()
stream.close()
p.terminate()

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


*** Recording has started - Keep quiet and only play the instrument ***
89 / 0.22714072465896606
84 / 0.0
85 / 0.0
84 / 0.0
83 / 0.0
*** Ctrl+C pressed ***
*** Recording has ended ***
